In [31]:
import copy
from time import time
from pathlib import Path

import torch
import tltorch
from torchsummary import summary
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tddl.trainer import Trainer


In [ ]:
# Load model
model = torch.load("/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/1625154185/model_52")


In [7]:
print(model)

ModifiedVGG16Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, paddin

In [10]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [17]:

conv = model.features[19]
print(conv)

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [34]:
factorization = 'cp'

fact_conv = tltorch.FactorizedConv.from_conv(conv, rank=0.5, decompose_weights=False, factorization=factorization)
# print(fact_conv)

fact_model = copy.deepcopy(model)
fact_model.features[19] = fact_conv
print(fact_model)

epochs = 10

logdir="/home/jetzeschuurman/gitProjects/phd/tddl/artifacts"

t = round(time())
logdir = Path(logdir).joinpath(str(t))
save = {
    "save_every_epoch": 1,
    "save_location": str(logdir),
    "save_best": True,
    "save_final": True,
    "save_model_name": "fact_model_conv_19_"+factorization
}

train_path = "/bigdata/dogs-vs-cats/train/"
valid_path = "/bigdata/dogs-vs-cats/valid/"

writer = SummaryWriter(log_dir=logdir.joinpath('runs'))

optimizer = optim.SGD(model.classifier.parameters(), lr=0.0001, momentum=0.99)
trainer = Trainer(train_path, valid_path, fact_model, optimizer, writer, save=save)

train_acc = trainer.test(loader="train")
print(f'{train_acc = }')
valid_acc = trainer.test()
print(f'{valid_acc = }')

trainer.train(epochs=epochs)

writer.close()

  0%|          | 0/625 [00:00<?, ?it/s]

ModifiedVGG16Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, paddin

  0%|          | 0/63 [00:00<?, ?it/s]

train_acc = 0.5


  0%|          | 0/63 [00:00<?, ?it/s]

valid_acc = 0.5



100%|██████████| 63/63 [00:11<00:00,  5.34it/s, loss=0.717]
64it [00:11,  5.31it/s, loss=0.717]                        
64it [00:12,  5.31it/s, loss=0.736]
65it [00:12,  5.48it/s, loss=0.736]
65it [00:12,  5.48it/s, loss=0.738]
66it [00:12,  5.41it/s, loss=0.738]
66it [00:12,  5.41it/s, loss=0.704]
67it [00:12,  5.53it/s, loss=0.704]
67it [00:12,  5.53it/s, loss=0.725]
68it [00:12,  5.44it/s, loss=0.725]
68it [00:12,  5.44it/s, loss=0.73] 
69it [00:12,  5.53it/s, loss=0.73]
69it [00:13,  5.53it/s, loss=0.731]
70it [00:13,  5.46it/s, loss=0.731]
70it [00:13,  5.46it/s, loss=0.695]
71it [00:13,  5.58it/s, loss=0.695]
71it [00:13,  5.58it/s, loss=0.66] 
72it [00:13,  5.44it/s, loss=0.66]
72it [00:13,  5.44it/s, loss=0.707]
73it [00:13,  5.57it/s, loss=0.707]
73it [00:13,  5.57it/s, loss=0.668]
74it [00:13,  5.47it/s, loss=0.668]
74it [00:13,  5.47it/s, loss=0.69] 
75it [00:13,  5.58it/s, loss=0.69]
75it [00:14,  5.58it/s, loss=0.678]
76it [00:14,  5.49it/s, loss=0.678]
76it [00:14,  5.49

In [36]:
echo = 'test'